In [1]:
import numpy as np 
from astropy.io import fits
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib ipympl

In [10]:
dgroup = np.loadtxt('../odata/CLAUDS_HSC_iband_group')
d2 = np.loadtxt('../odata/iCLAUDS_HSC_iband_2')

In [14]:
digal = np.loadtxt('../odata/CLAUDS_HSC_iband_igal')
d1 = np.loadtxt('../odata/iCLAUDS_HSC_iband_1')

In [3]:
digal0 = np.loadtxt('../odata/CLAUDS_HSC_iband_igal0')

In [4]:
def load_maskid(zr):
    if zr == 4:
        clu_maskid = np.array([305629, 365910, 366011, 146894, 165542, 238352, 243075, 245552, 261864, 263361, 266999, 305574, 
307591, 315030, 320952, 320956, 338677, 339214, 344676, 344727, 347891, 347893, 349194, 350083, 355748,
356972, 357238, 362410, 364580, 375231, 375292, 377484, 381933, 382681, 383192, 396768, 396787, 399071,
400843, 166802, 202392, 282594, 315439, 338392, 346168, 365910, 366011, 399440], dtype = np.int64)
#         clu_maskid = np.array([202392, 266999, 338392, 346168, 347893, 365910, 364580, 366011, 396768, 396787, 399440], dtype = np.int64) #for z >= 4, Ng >= 3
    elif zr == 3:
        clu_maskid = np.array([31967,53759,83025,99101,101901,104112,113992,111646,125027,129803,127524,132446,
                            134943, 135168, 137908, 138114, 138430, 138467, 139829, 141254, 146894, 149069, 152231, 152511, 158891, 161521, 160824, 163470, 165943, 166952, 167037, 167168,
                            170162, 171558, 172497, 173225, 174364, 172776, 174581, 174844, 175135, 176211, 176721,
                            176804, 177948, 178357, 177802, 181656, 182093, 182691, 184252, 187638, 185304,
                            189728, 193002, 195538, 198820, 199343], dtype = np.int64) # z >= 3; Ng >= 5
    else:
        clu_maskid = np.array([5523, 16745, 19964, 20765, 19829, 19964, 21166, 22746, 22845, 23796,
                              24780, 24925, 25560, 26144, 25811, 27009, 27140, 28660, 29981, 
                              31490, 31532, 31708, 32220, 32987, 33629, 33668, 33842, 34135, 34991, 35881,
                              35803, 38865, 38949, 39659, 40048, 41118, 41181, 41487, 42115,
                              41681, 42537, 43088, 43360, 43827, 43486, 44349, 44458, 45425,
                              46167, 46174, 46198, 46281, 46932, 47024, 46979, 47015, 47255,
                              47978, 48581, 49181, 49538, 49779, 50196, 50213, 50397, 50753,
                              51261, 51640, 51485, 52641, 53196, 53477, 53510, 53352, 53425,
                              53759, 54006, 53954, 54142, 55694, 55706, 55714, 56078, 56814,
                              57006, 57164, 57308, 57387, 57520, 57657, 57666, 57745, 57803,
                              57827, 57862, 58461, 58488, 58819, 58625, 58870, 58959, 59247,
                              59527, 59841, 59906, 60339, 60504, 60581, 60586, 61076, 61063,
                              61182, 61601, 61763, 62516, 62602, 62380, 62353, 62761, 62771,
                              62810, 62839, 62997, 63090, 63268, 63287, 63986, 64294, 64224,
                              64874, 64985, 65219, 65364, 65468, 65498, 65412, 65543, 65874,
                              66316, 66348, 66363, 66475, 66696, 66817, 66837, 67322, 67525,
                              67989, 68018, 68101, 68399, 68442, 68526, 68688, 68655], dtype = np.int64) # z >= 2; Ng >= 10
        
    return clu_maskid

In [5]:
def cal_Mnei(gal_id):
    from scipy import stats
    Ngal = gal_id.shape[0]
    idclu = np.zeros(Ngal, dtype = np.int64)
    Mnei = 0
    for i in range(Ngal):
        gid = gal_id[i]
        Lgal = 10**(0.4 * (4.85 - digal[gid-1,-1]) - 10)
        if Lgal > 50:
            Lgal = 50
        
        grpid = np.int64(d1[gid-1,1])
        Mgrp0 = 10**dgroup[grpid-1,-2]
        Lgrp0 = dgroup[grpid-1,-1]

        Mnei += Lgal/Lgrp0 * Mgrp0
    
    return Mnei


In [6]:
def cal_rp(ag_redz,c_redz,ag_ra,ag_dec,c_ra,c_dec):
    from astropy.cosmology import FlatLambdaCDM
    from astropy import units as u
    from astropy.coordinates import SkyCoord
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)
    
    dr_gal = cosmo.comoving_distance(ag_redz).value * 0.674 #Mpc/h
    dr_clu = cosmo.comoving_distance(c_redz).value * 0.674 #Mpc/h

    cgal = SkyCoord(ra=ag_ra*u.degree, dec=ag_dec*u.degree, distance=dr_gal)
    cclu = SkyCoord(ra=c_ra*u.degree, dec=c_dec*u.degree, distance=dr_clu)
    cgal_x = cgal.cartesian.x
    cgal_y = cgal.cartesian.y
    cgal_z = cgal.cartesian.z

    cclu_x = cclu.cartesian.x
    cclu_y = cclu.cartesian.y
    cclu_z = cclu.cartesian.z

    l = np.array([cgal_x+cclu_x, cgal_y+cclu_y, cgal_z+cclu_z]).T / 2
    s = np.array([cclu_x - cgal_x, cclu_y - cgal_y, cclu_z - cgal_z]).T
    r_pi = np.sum(l*s,axis = 1) / np.sqrt(np.sum(l**2, axis = 1)) 
    r_p = np.sqrt(np.sum(s**2,axis = 1) - r_pi**2)
    
    return r_p

In [23]:
def cal_Mnei_Nnei(dz,sel_clu,rci):
    from astropy.coordinates import SkyCoord
    from astropy.cosmology import FlatLambdaCDM
    from astropy import units as u
    cosmo = FlatLambdaCDM(H0=67.4, Om0=0.315)

    clu_ra = dgroup[sel_clu,2]
    clu_dec = dgroup[sel_clu,3]
    clu_redz = dgroup[sel_clu,4]
    idx_gal = np.where(d2[:,0] == (sel_clu+1))[0]
    galid = np.int64(d2[idx_gal,1])
    
    idx_region = np.where((np.abs(clu_redz - digal[:,3]) < dz) & 
                          (np.abs(clu_ra - digal[:,1]) < 1) & 
                          (np.abs(clu_dec - digal[:,2]) < 1))[0] 
    
    idx_around = list(idx_region)
#     gglistid = list(galid-1)
#     for nn in range(galid.shape[0]):
#         if gglistid[nn] in idx_around:
#             idx_around.remove(gglistid[nn])
    gal_id = np.int64(digal[idx_around, 0])
    gal_ra = digal[idx_around, 1]
    gal_dec = digal[idx_around, 2]
    gal_redz = digal[idx_around, 3]
    
    d_r = cal_rp(gal_redz,clu_redz,gal_ra,gal_dec,clu_ra,clu_dec)
    
#     d_A = cosmo.comoving_distance(z=clu_redz) * 0.674 #Mpc/h
#     cgal = SkyCoord(gal_ra, gal_dec, unit="deg")
#     cclu = SkyCoord(clu_ra, clu_dec, unit="deg")
#     sep = cgal.separation(cclu)
#     d_r0 = (sep * d_A).to(u.Mpc, u.dimensionless_angles()) #to be a comoving distance
#     idx_nei0 = np.where(d_r0.value < 6)[0]
    
    idx_nei = np.where(d_r < rci)[0]
    Mnei = cal_Mnei(gal_id[idx_nei])
    Nnei = idx_nei.shape[0]
    
#     print(idx_nei0.shape, idx_nei.shape)
    return Mnei, Nnei, gal_id[idx_nei]

In [78]:
rci = 5
gid = [47826, 20048, 68057] #z ~ 2

rci = 6
gid = [192242, 104820, 186417] #z ~ 3

rci = 7
gid = [321925, 354967, 297062] #z ~ 4
for i in range(3):
    Mnei,Nnei = cal_Mnei_Nnei(0.1,gid[i]-1,rci)
    print(Nnei)

61
23
3


In [79]:
def cal_MN(dz,zr,nm,rci):
    if zr >= 4:
        sel_redz = np.where((dgroup[:,4] >= zr) & (dgroup[:,1] >= nm))[0]
    else:
        sel_redz = np.where((dgroup[:,4] >= zr) & (dgroup[:,4] < zr + 1) & (dgroup[:,1] >= nm))[0]
    clu_id = np.int64(dgroup[sel_redz, 0])
    print('Before exclude Ngrp:',clu_id.shape)

    #exclude the influence of masks
    clu_id = list(clu_id)
    clu_maskid = load_maskid(zr)
    for nn in range(clu_maskid.shape[0]):
        if clu_maskid[nn] in clu_id:
            clu_id.remove(clu_maskid[nn])
    clu_id = np.array(clu_id, dtype = np.int64)
    print('After exclude Ngro:',clu_id.shape)
    Ngroup = clu_id.shape[0]

    Mtot = np.zeros(Ngroup)
    Ntot = np.zeros(Ngroup)
    neighb = []
    PCid = []
    for i in range(Ngroup):
        Mtot[i], Ntot[i], neighbGalID = cal_Mnei_Nnei(dz, clu_id[i]-1,rci)
        neighb.extend(neighbGalID) #record neighboring galaxy id
        PCid.extend(np.tile(i+1, neighbGalID.shape[0])) #record repeated PC ID
    #     print(Mtot[i]/1e14, Ntot[i])
        if i%100 == 0:
            print(i)
    return Mtot, Ntot, clu_id, neighb, PCid

In [80]:
dz = 0.1; zr = 4; nm = 3; rci = 7;
Mtot4,Ntot4,clu_id4,neighb4,PCid4 = cal_MN(dz,zr,nm,rci)

Before exclude Ngrp: (89,)
After exclude Ngro: (43,)
0


In [26]:
dz = 0.1; zr = 3; nm = 5;rci = 6;
Mtot3,Ntot3,clu_id3,neighb3,PCid3 = cal_MN(dz,zr,nm,rci)

Before exclude Ngrp: (400,)
After exclude Ngro: (343,)
0
100
200
300


In [81]:
dz = 0.1; zr = 2; nm = 10;rci = 5;
Mtot2,Ntot2,clu_id2,neighb2,PCid2 = cal_MN(dz,zr,nm,rci)

Before exclude Ngrp: (914,)
After exclude Ngro: (761,)
0
100
200
300
400
500
600
700


In [74]:
##select most rich and poor galaxies
idx_4max = np.argmax(Ntot4)
idx_4min = np.argsort(Ntot4)[0]
print("At z ~ 4: Nmax id and Nmin id: ", clu_id4[idx_4max], clu_id4[idx_4min])
print(dgroup[68057-1])

idx_3max = np.argmax(Ntot3)
idx_3min = np.argsort(Ntot3)[2]
print("At z ~ 3: Nmax id and Nmin id: ", clu_id3[idx_3max], clu_id3[idx_3min])

idx_2max = np.argmax(Ntot2)
idx_2min = np.argsort(Ntot2)[2]
print("At z ~ 2: Nmax id and Nmin id: ", clu_id2[idx_2max], clu_id2[idx_2min])

At z ~ 4: Nmax id and Nmin id:  321925 297062
[6.805700e+04 1.000000e+01 1.496482e+02 3.678300e+00 2.160900e+00
 1.331330e+01 2.878390e+01]
At z ~ 3: Nmax id and Nmin id:  192242 186417
At z ~ 2: Nmax id and Nmin id:  47826 68057


In [18]:
plt.figure(figsize = (4.0,3.7))
# plt.figure(figsize = (5.3,4.0))
import matplotlib as mpl
mpl.rcParams['axes.linewidth'] = 1.2 #set the value globally

def fit_points(Ntot,Mtot):
    def linear_dis(xx, a, k):
        return a + xx * k
    from scipy.optimize import curve_fit
    pp, pcov = curve_fit(linear_dis, np.log10(Ntot), np.log10(Mtot), p0 = (11,1))
    print(pp)
    xx = np.linspace(0,3,100)
    yy = linear_dis(xx, pp[0], pp[1])
    return xx, yy

plt.plot(Ntot2, Mtot2, 'o', ms = 2, alpha = 0.8, label = r'$\mathrm{S}1: 2 \leq z < 3, N_{\rm g} \geq 10$')
xx,yy = fit_points(Ntot2,Mtot2)
# plt.plot(10**xx,10**yy, c = 'C0', zorder = -1, alpha = 0.7, ls = '--')
plt.plot(Ntot3, Mtot3, 'o', ms = 2, alpha = 0.8, c = 'orange',label = r'$\mathrm{S}2: 3 \leq z < 4, N_{\rm g} \geq 5$')
xx,yy = fit_points(Ntot3,Mtot3)
# plt.plot(10**xx,10**yy, c = 'orange', zorder = -1, alpha = 0.7, ls = '--')
plt.plot(Ntot4, Mtot4, 'o', ms = 2, alpha = 0.8, c = 'k', label = r'$\mathrm{S}3: z \geq 4, N_{\rm g} \geq 3$')
xx,yy = fit_points(Ntot4,Mtot4)
# plt.plot(10**xx,10**yy, c = 'k', zorder = -1, alpha = 0.7, ls = '--')
# p_spec10, pcov = curve_fit(linear_dis, Mtot4, Ntot4, p0 = (100, 0.27, 0.1))

plt.ylabel(r'$M_{\rm nei}[h^{-1}\rm M_{\odot}]$', fontsize = 12)
plt.xlabel(r'$N_{\rm nei}$', fontsize = 12)
plt.axhline(1e14, ls = '--', color = 'r', lw = 1)

plt.loglog()
plt.ylim(1e12, 6e14)
plt.xlim(2, 6e2)
plt.legend(loc = 4)
plt.tick_params(top = 'on', right = 'on', which='both',direction = 'in',labelsize = 10)

plt.tight_layout()
# plt.savefig('../figs/Mnei_Nnei.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[12.06612746  0.96761374]
[11.93583508  0.98058993]
[11.81445246  1.07180266]


In [82]:
#save protocluster catalogue data

#S1
nPC = np.arange(761)+1
ra_S1 = dgroup[clu_id2-1,2]
dec_S1 = dgroup[clu_id2-1,3]
redz_S1 = dgroup[clu_id2-1,4]
dS1 = np.vstack((nPC, clu_id2, ra_S1, dec_S1, redz_S1, Ntot2, np.log10(Mtot2))).T
np.savetxt('/home/qyli/clu_finder/CLAUDS_HSC/catalogue/ReleaseSample/PC/PC_S1', dS1,
           fmt = '%4d   %8d   %8.4f   %8.4f   %.4f   %4d   %8.4f')

dS1_neb = np.vstack((PCid2, neighb2)).T
np.savetxt('/home/qyli/clu_finder/CLAUDS_HSC/catalogue/ReleaseSample/PC/iPC_gal_S1', dS1_neb,
           fmt = '%4d   %d')

#S2
nPC = np.arange(343)+1
ra_S2 = dgroup[clu_id3-1,2]
dec_S2 = dgroup[clu_id3-1,3]
redz_S2 = dgroup[clu_id3-1,4]
dS2 = np.vstack((nPC, clu_id3, ra_S2, dec_S2, redz_S2, Ntot3, np.log10(Mtot3))).T
np.savetxt('/home/qyli/clu_finder/CLAUDS_HSC/catalogue/ReleaseSample/PC/PC_S2', dS2,
           fmt = '%4d   %8d   %8.4f   %8.4f   %.4f   %4d   %8.4f')

dS2_neb = np.vstack((PCid3, neighb3)).T
np.savetxt('/home/qyli/clu_finder/CLAUDS_HSC/catalogue/ReleaseSample/PC/iPC_gal_S2', dS2_neb,
           fmt = '%4d   %d')

#S3
nPC = np.arange(43)+1
ra_S3 = dgroup[clu_id4-1,2]
dec_S3 = dgroup[clu_id4-1,3]
redz_S3 = dgroup[clu_id4-1,4]
dS3 = np.vstack((nPC, clu_id4, ra_S3, dec_S3, redz_S3, Ntot4, np.log10(Mtot4))).T
np.savetxt('/home/qyli/clu_finder/CLAUDS_HSC/catalogue/ReleaseSample/PC/PC_S3', dS3,
           fmt = '%4d   %8d   %8.4f   %8.4f   %.4f   %4d   %8.4f')

dS3_neb = np.vstack((PCid4, neighb4)).T
np.savetxt('/home/qyli/clu_finder/CLAUDS_HSC/catalogue/ReleaseSample/PC/iPC_gal_S3', dS3_neb,
           fmt = '%4d   %d')

In [20]:
print(np.sum())

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42])

## test code

In [ ]:
# #This is calculate mean value of high-redshift groups after excluding contamination
# #contamination groups id at z > 5
# gid5 = [570541,624159] #5.2 < z < 5.6
# #contamination groups id at z > 4
# gid4 = [166802, 415510, 464146, 555750, 621690, 622970, 638654, ] #4.8 < z < 5.2

# sel_redz = np.where((dgroup[:,4] >= 5) & (dgroup[:,1] >= 2))[0]
# clu_id = np.int64(dgroup[sel_redz, 0])
# print('Before exclude Ngrp:',clu_id.shape)

# #exclude the influence of masks
# clu_id = list(clu_id)
# clu_maskid = np.array(gid5, dtype = np.int64)
# for nn in range(clu_maskid.shape[0]):
#     if clu_maskid[nn] in clu_id:
#         clu_id.remove(clu_maskid[nn])
# clu_id = np.array(clu_id, dtype = np.int64)

# mean_Mh = np.median(10**dgroup[clu_id-1, -2])
# mean_r180 = np.median(0.781*(10**dgroup[clu_id-1, -2]/1e14/0.315)**(1/3))
# print(mean_Mh/1e12, mean_r180)

In [ ]:
# def cal_Mnei(gal_id):
#     from scipy import stats
#     Ngal = gal_id.shape[0]
#     Mnei = np.zeros(Ngal)
#     idclu = np.zeros(Ngal, dtype = np.int64)
#     Nm0 = 0
#     for i in range(Ngal):
#         gid = gal_id[i]
#         grpid = np.int64(d1[gid-1,1])
#         Mnei[i] = 10**dgroup[grpid-1,-2]
#         idclu[i] = np.int64(dgroup[grpid-1,0])
#         Nm0 += dgroup[grpid-1,1]
    
#     overN = stats.find_repeats(idclu).counts 
#     overV = stats.find_repeats(idclu).values
#     overV = np.int64(overV)
# #     print(overN)
#     Nm00 = Nm0 - np.sum(dgroup[overV-1,1] * (overN - 1))
#     Mnei0 = np.sum(Mnei) -  np.sum(10**dgroup[overV-1,-2] * (overN-1))
    
#     print(Ngal, Nm0, Nm00)
#     return Mnei0

In [ ]:
# idclu = 315030
# idx_gal = np.where(d2[:,0] == idclu)[0]
# galid = np.int64(d2[idx_gal,1])
# Msunbj = 4.85
# grplum0 = 10**(0.4 * (Msunbj - digal[galid-1,-1]) - 10)
# idxlum = np.where(grplum0 > 50)[0]
# grplum0[idxlum] = 50
# grplum0 = np.sum(grplum0)
# print(np.sum(digal0[galid-1,-1]), dgroup[idclu-1,-1], grplum0)